# Demo: Delta Live Tables (DLT) com Yellow Taxi

Este notebook demonstra como utilizar o **Delta Live Tables (DLT)** no Databricks para criar pipelines de ingestão e transformação de dados confiáveis, auditáveis e fáceis de manter. O DLT automatiza tarefas como gerenciamento de qualidade, versionamento e monitoramento dos dados.

Vamos usar o dataset do yellow taxi, já utilizado no teste do Auto Loader, para mostrar como criar tabelas bronze (raw), silver (limpas) e gold (agregadas) com DLT.

## O que é Delta Live Tables (DLT)?

O Delta Live Tables é uma framework de pipelines declarativos da Databricks que permite:

- Declarar tabelas e transformações como código Python ou SQL.
- Gerenciar automaticamente a qualidade dos dados (expectations).
- Versionar e monitorar pipelines de dados.
- Automatizar ingestão, transformação e atualização de tabelas Delta.

DLT é ideal para pipelines de ingestão incremental, ETL e preparação de dados para analytics e machine learning.

## Pipeline DLT: Yellow Taxi

Vamos criar um pipeline DLT com três camadas:

- **Bronze:** ingestão bruta dos dados Parquet do yellow taxi.
- **Silver:** limpeza e padronização dos dados.
- **Gold:** agregação de métricas para análise.

In [0]:
-- Bronze layer: Raw data ingestion
CREATE OR REFRESH STREAMING TABLE taxi_raw_records 
(CONSTRAINT valid_distance EXPECT (trip_distance > 0.0) ON VIOLATION DROP ROW)
AS SELECT *
FROM STREAM(samples.nyctaxi.trips);

-- Silver layer 1: Flagged rides
CREATE OR REFRESH STREAMING TABLE flagged_rides 
AS SELECT
  date_trunc("week", tpep_pickup_datetime) as week,
  pickup_zip as zip, 
  fare_amount, trip_distance
FROM
  STREAM(LIVE.taxi_raw_records)
WHERE ((pickup_zip = dropoff_zip AND fare_amount > 50) OR
       (trip_distance < 5 AND fare_amount > 50));

-- Silver layer 2: Weekly statistics
CREATE OR REFRESH MATERIALIZED VIEW weekly_stats
AS SELECT
  date_trunc("week", tpep_pickup_datetime) as week,
  AVG(fare_amount) as avg_amount,
  AVG(trip_distance) as avg_distance
FROM
 live.taxi_raw_records
GROUP BY week
ORDER BY week ASC;

-- Gold layer: Top N rides to investigate
CREATE OR REPLACE MATERIALIZED VIEW top_n
AS SELECT
  weekly_stats.week,
  ROUND(avg_amount,2) as avg_amount, 
  ROUND(avg_distance,3) as avg_distance,
  fare_amount, trip_distance, zip 
FROM live.flagged_rides
LEFT JOIN live.weekly_stats ON weekly_stats.week = flagged_rides.week
ORDER BY fare_amount DESC
LIMIT 3;

## Como executar este notebook

1. Crie um pipeline Delta Live Tables no Databricks e aponte para este notebook.
2. Execute o pipeline para ver as tabelas bronze, silver e gold sendo criadas e atualizadas automaticamente.
3. Use as tabelas gold para análises e dashboards.

**DLT** garante qualidade, rastreabilidade e automação no seu pipeline de dados!

In [0]:
select * from lab.taxi.taxi_raw_records limit 5